In [2]:
# reddit crawler
import pandas as pd
import praw
from LSTM_predict import *

your_client_id='ElyKc6o3du1IMb5LP2HYjg'
your_client_secret='YimfVn3bTLyVFu_XkJuLxrNkR3vHAQ'
your_user_name='guntsv'

reddit = praw.Reddit(client_id=your_client_id,
                     client_secret=your_client_secret,
                     user_agent=your_user_name,
                     check_for_async=False)

subreddit = reddit.subreddit('bitcoin')
topics = [*subreddit.top(limit=50)] # top posts all time
print(len(topics))

title = [n.title for n in topics]
df_topics = pd.DataFrame({"title": title})

df_result = pd.DataFrame(prediction(df_topics['title']))
df_result = df_result.rename(columns={0: 'Title', 1: 'Rating'})
df_result.head()

50


,Title,Rating
0,"It's official! 1 Bitcoin = $10,000 USD",2
1,The last 3 months in 47 seconds.,1
2,Tesla buys $1.5b in Bitcoin and is looking to ...,3
3,It's over 9000!!!,3
4,Everyone who's trading BTC right now,2


In [3]:
from collections import Counter
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
nlp = spacy.load('en_core_web_md')
# ruler = nlp.add_pipe("entity_ruler")
# ruler.from_disk(skill_path)
# nlp.pipe_names

In [4]:
def preprocessing(sentence):

    stopwords = list(STOP_WORDS)
    doc = nlp(sentence)
    cleaned_tokens = []

    for token in doc:
        if token.text not in stopwords and token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and \
                token.pos_ != 'SYM':
            cleaned_tokens.append(token.lemma_.lower().strip())

    return " ".join(cleaned_tokens)

In [12]:
df_result['clean'] = df_result['Title'].apply(preprocessing)
df_result['clean'].head()

0                   it official 1 bitcoin = 10,000 usd
1                                the 3 month 47 second
2    tesla buy 1.5b bitcoin look accept crypto form...
3                                              it 9000
4                             everyone trade btc right
Name: clean, dtype: object

In [16]:
#1. tokenize
#data cleaned
corpus = df_result['clean']
#data tokenized
corpus_tokenized = [sent.split(" ") for sent in corpus]
#2. numericalize (vocab)
#2.1 get all the unique words
#we want to flatten unit (basically merge all list)
flatten = lambda l: [item for sublist in l for item in sublist]
vocabs = list(flatten(corpus_tokenized))

#2.2 assign id to all these vocabs
word2index = {v: idx for idx, v in enumerate(vocabs)}

#adding unknown word
vocabs.append('<UNK>')
word2index['<UNK>'] = len(vocabs) - 1

voc_size = len(vocabs)
voc_size

317

In [18]:
word_freq = Counter(vocabs)
common_words = word_freq.most_common(10)
print (common_words)
##All Blend

[('bitcoin', 23), ('i', 8), ('buy', 7), ('it', 4), ('the', 4), ('btc', 3), ('robinhood', 3), ('this', 3), ('sell', 3), ('new', 3)]
